In [1]:
!pip install huggingface-hub
!pip install datasets
!pip install trl
!pip install peft
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install flash-attn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 12.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 15.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 433.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 441.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 505.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 332.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
# Import the 'login' function from the huggingface_hub library to authenticate and access your Hugging Face account.
from huggingface_hub import login
# Call the login function to authenticate. You'll need to enter your credentials or token.
login(token='')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/jovyan/.cache/huggingface/token
Login successful


In [3]:
import torch
import os
import sys
import json
import pandas as pd
from IPython.display import display, HTML
from datetime import datetime
import datasets
from datasets import load_dataset

import torch
import trl
from trl import setup_chat_format,SFTTrainer

import peft
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)

from random import randint
print("Torch version:", torch.__version__)
print("Datasets version:", datasets.__version__)
print("TRL (Transformers Reinforcement Learning) version:", trl.__version__)
print("PEFT (Parameter Efficient Fine-Tuning) version:", peft.__version__)
print("Transformers version:", transformers.__version__)

2024-06-09 15:34:04.227429: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-09 15:34:04.267878: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-09 15:34:04.921489: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Torch version: 2.2.2+cu121
Datasets version: 2.19.2
TRL (Transformers Reinforcement Learning) version: 0.9.4
PEFT (Parameter Efficient Fine-Tuning) version: 0.11.1
Transformers version: 4.41.2


In [23]:
# Define the identifier for the model to be loaded, specifically "meta-llama/Meta-Llama-3-8B".
model_id = "meta-llama/Meta-Llama-3-8B" 

# Configure the model for low-precision (4-bit) quantization to reduce memory usage and potentially increase inference speed.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable loading the model in 4-bit precision
    bnb_4bit_use_double_quant=True,  # Use double quantization for better precision handling
    bnb_4bit_quant_type="nf4",  # Set the quantization type to 'nf4', a specific 4-bit format
    bnb_4bit_compute_dtype=torch.bfloat16  # Use bfloat16 as the datatype for computation to balance performance and accuracy
)

# Load the pre-trained causal language model from Hugging Face's model hub.
# The model is automatically distributed across available GPUs if possible using `device_map="auto"`.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",  # Automatically distribute the model across available GPUs
    attn_implementation="flash_attention_2",  # Use an optimized attention mechanism for better performance
    torch_dtype=torch.bfloat16,  # Use bfloat16 as the default tensor data type for all model parameters
    quantization_config=bnb_config  # Apply the defined quantization configuration
)

# Load the tokenizer associated with the model, enabling fast tokenization.
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.padding_side = 'right'  # Set padding to the right to align with model's expectations and prevent warnings

# Redefine the pad_token and pad_token_id to use the out-of-vocabulary token (unk_token), 
# which can help in handling tokens that are not in the tokenizer's vocabulary during processing.
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# Set up the chat format for the model using a predefined chat template (ChatML) designed for OpenAI API interactions.
# This step is useful for ensuring the model and tokenizer are properly configured for generating responses in a chat environment.
model, tokenizer = setup_chat_format(model, tokenizer)


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [15]:
model

NameError: name 'model' is not defined

In [39]:
model.save_pretrained("model", from_pt=True) 